In [1]:
using Pkg
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("Gurobi")
# Pkg.add("JuMP")
# Pkg.add("LinearAlgebra")
# Pkg.add("Plots")
# Pkg.add("Clustering")
# Pkg.add("StableRNGs")

In [2]:
using CSV, DataFrames, Gurobi, JuMP, LinearAlgebra, Plots, Clustering, StableRNGs, Random, Statistics

In [51]:
function read_data(file, scaling)
    # normalized_root = "./normalized_data"
    # scaled_root = "./scaled_data"
    
    if scaling == "normalized"
        data = CSV.read("./normalized_data/$file", DataFrame)
        for c in names(data, Any)
            data[!, c]= Float64.(data[!, c])
        end
        return data
    elseif scaling == "processed"
        data = CSV.read("./processed_data/$file", DataFrame)
        for c in names(data, Any)
            data[!, c]= Float64.(data[!, c])
        end
        return data
    else
        data = CSV.read("./scaled_data/$file", DataFrame)
        for c in names(data, Any)
            data[!, c]= Float64.(data[!, c])
        end
        return data
    end
end

function k_medoids_cluster(data, k, seed=0)
    Random.seed!(seed)
    data_matrix = Matrix(data)
    distance_matrix = zeros(size(data_matrix)[1], size(data_matrix)[1])
    
    for i in 1:size(data_matrix)[1]
        for j in 1:size(data_matrix)[1]
            distance_matrix[i, j] = norm(data_matrix[i, :] .- data_matrix[j, :])^2
        end
    end
    
    kmedoids_result = kmedoids(distance_matrix, k)
    # kmedoids_result.totalcost (within-cluster sum of square distance)
    # a = assignments(kmeans_result) # get the assignments of points to clusters
    # c = counts(kmeans_result) # get the cluster sizes
    # M = kmeans_result.medoids # get index number of medoids observations (ordered from k=1, ..., k=K
    return kmedoids_result
end

k_medoids_cluster (generic function with 2 methods)

In [4]:
abalone = read_data("abalone_z.csv", "normalized")
similarity_prediction = read_data("similarity_prediction_z.csv", "normalized")

abalone_raw = read_data("abalone.csv", "processed")
similarity_prediction_raw = read_data("similarity_prediction.csv", "processed")

abalone_scaled = read_data("abalone_mm.csv", "scaled")
similarity_prediction_scaled = read_data("similarity_prediction_mm.csv", "scaled");

Row,tanimoto_cdk_Extended,TanimotoCombo,pchembl_distance,target_name_5HT2B,target_name_CYP2D6,target_name_HERG,simil_2D,simil_3D,dissimil_2D,dissimil_3D,pair_type_dis2D_dis3D,pair_type_dis2D_sim3D,pair_type_sim2D_dis3D,pair_type_sim2D_sim3D,n_answers,n_similar,frac_similar
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.472838,1.0,0.0930931,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.578947,0.72,0.818182
2,0.430275,0.866106,0.102102,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.263158,0.36,0.5625
3,0.451161,0.863519,0.147147,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.526316,0.32,0.380952
4,0.463027,0.854463,0.39039,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.473684,0.6,0.75
5,0.333648,0.849935,0.15015,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.631579,0.6,0.652174
6,0.544889,0.849935,0.432432,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.315789,0.28,0.411765
7,0.351486,0.846701,0.027027,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.210526,0.48,0.8
8,0.417813,0.815653,0.0600601,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.947368,0.76,0.655172
9,0.254746,0.796248,0.12012,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.32,0.727273


In [69]:
function kmedoids_optimal(data, num_rows, distance, k_max, time_limit, warm_start, seed=1)

    data = Matrix(data[1:num_rows, :])
    n = size(data)[1]
    p = size(data)[2]
    
    observation_range = 1:n
    variable_range = 1:p
    cluster_range = 1:k_max
    big_M = 1e3

    # t = time()
    model = Model(optimizer_with_attributes(Gurobi.Optimizer, "TimeLimit" => time_limit, "Seed" => seed, "OutputFlag" => 0))

    @variable(model,
        m[i in observation_range, k in cluster_range], Bin)
    @variable(model,
        z[i in observation_range, k in cluster_range], Bin)
    @variable(model,
        c[k in cluster_range, j in variable_range])
    @variable(model,
        d[i in observation_range, j in variable_range] >= 0)

    @constraint(model,
        must_have_cluster[i in observation_range],
        sum( (z[i, k]) for k in cluster_range) == 1)
    @constraint(model,
        must_have_medoid[k in cluster_range],
        sum( (m[i, k]) for i in observation_range) == 1)
    @constraint(model,
        medoid_restriction_1[i in observation_range],
        sum( (m[i, k]) for k in cluster_range) <= 1)
    @constraint(model,
        medoid_restriction_2[i in observation_range, k in cluster_range],
        m[i, k] <= z[i, k])
    
    @constraint(model,
        medoid_selection_UB[i in observation_range, j in variable_range, k in cluster_range],
        c[k, j] <= data[i, j] + big_M*(1 - m[i, k]))
    @constraint(model,
        medoid_selection_LB[i in observation_range, j in variable_range, k in cluster_range],
        data[i, j] - big_M*(1 - m[i, k]) <= c[k, j])

    if warm_start == true
        data_kmedoids = k_medoids_cluster(data, k_max, seed)
        heur_medoids = data_kmedoids.medoids

        m_ik = Int64.(zeros(size(data)[1], k_max))
        z_ik = Int64.(zeros(size(data)[1], k_max))
        c_kj = zeros(k_max, size(data)[2])

        cluster = 0
        for index in heur_medoids
            cluster = cluster + 1

            set_start_value(m[index, cluster], 1)
            for j in variable_range
                set_start_value(c[cluster, j], data[index, j])
            end
        end
    
        for index in 1:size(assignments(data_kmedoids))[1]
            set_start_value(z[index, assignments(data_kmedoids)[index]], 1)
        end
    end

    if distance == "manhattan"
        @constraint(model,
            distance_1[i in observation_range, j in variable_range, k in cluster_range],
            d[i, j] >= data[i, j] - c[k, j] - big_M*(1 - z[i, k]))
        @constraint(model,
            distance_2[i in observation_range, j in variable_range, k in cluster_range],
            d[i, j] >= c[k, j] - data[i, j] - big_M*(1 - z[i, k]))
    else
        @constraint(model,
            distance_1[i in observation_range, j in variable_range, k in cluster_range],
            d[i, j] >= (data[i, j] - c[k, j])^2 - big_M*(1 - z[i, k]))
    end

    @objective(model,
        Min,
        sum( sum( (d[i, j]) for j in variable_range) for i in observation_range))

    optimize!(model)
    # dt = time() - t
    # println()
    # println("Running Optimal KMedoids for following args:")
    # println("num_rows=$num_rows, distance=$distance, k_max=$k_max, time_limit=$time_limit, warm_start=$warm_start, seed=$seed")
    # println("Objective function value: $(JuMP.objective_value(model)))")
    # println("Solved in $(round(dt, digits=3)) seconds")
    # println("--------------------------------------------------------------")

    return model
end

kmedoids_optimal (generic function with 3 methods)

In [6]:
function get_assignments(model)
    z_values = value.(model[:z])  
    n, k_max = size(z_values)
    assignments = zeros(Int, n)

    for i in 1:n
        for k in 1:k_max
            if z_values[i, k] > 0.5  
                assignments[i] = k
                break
            end
        end
    end

    return assignments
end

function get_centers(model)
    c_values = value.(model[:c])  
    return c_values
end



function evaluate_clustering(data, assignments, centers, distance)
    data_matrix = Matrix(data)
    K = maximum(assignments)
    n, p = size(data_matrix)

    wcssd = zeros(K)

    for k in 1:K
        cluster_obs = data_matrix[assignments .== k, :]
        for i in 1:size(cluster_obs)[1]
            if distance == "manhattan"
                wcssd[k] += sum(abs.(cluster_obs[i, :] .- centers[k, :]))^2
            else
                wcssd[k] += norm(cluster_obs[i, :] .- centers[k, :])^2
            end
        end
    end

    return sum(wcssd)
end


function average_silhouette_score(data, assignments, centers)
    data_matrix = Matrix(data)
    n = size(data_matrix, 1)
    K = maximum(assignments)

    # Function to compute the average distance to points in a given cluster
    function avg_distance_to_cluster(point, cluster_points)
        return mean([norm(point - other_point) for other_point in eachrow(cluster_points)])
    end

    silhouette_scores = zeros(n)

    for i in 1:n
        # Current point and its assigned cluster
        point = data_matrix[i, :]
        cluster = assignments[i]

        # a(i): Average distance to points in the same cluster
        same_cluster_points = data_matrix[assignments .== cluster, :]
        a_i = cluster == 1 ? 0 : avg_distance_to_cluster(point, same_cluster_points)

        # b(i): Minimum average distance to points in other clusters
        b_i = Inf
        for k in 1:K
            if k != cluster
                other_cluster_points = data_matrix[assignments .== k, :]
                b_i = min(b_i, avg_distance_to_cluster(point, other_cluster_points))
            end
        end

        # Silhouette score for point i
        silhouette_scores[i] = (b_i - a_i) / max(a_i, b_i)
    end

    return mean(silhouette_scores)
end


average_silhouette_score (generic function with 1 method)

# Metric Analysis

In [74]:
using DataFrames

function run_clustering_cases(datasets, raw_data, proportions, k_values, time_limits, distances)
    results = DataFrame()
    
     for (dataset_name, data) in datasets
        for prop in proportions
            num_rows = floor(Int, size(data)[1] * prop)
            for k in k_values
                for time_limit in time_limits
                    for dist in distances
                        # k_medoids_cluster(data, k, seed=0)
                        println(dataset_name, " ", prop, " ", k, " ", time_limit, " ", dist, " type=$(typeof(data[1:num_rows, :]))")
                        base_result = k_medoids_cluster(data[1:num_rows, :], k)
                        base_assignments = base_result.assignments
                        base_centers = Matrix(data[base_result.medoids, :])  #used to be: base_centers = base_result.centers'
                        base_wcss = evaluate_clustering(data[1:num_rows, :], base_assignments, base_centers, dist)
                        base_silhouette = average_silhouette_score(data[1:num_rows, :], base_assignments, base_centers)
                        
                        base_wcss_raw = evaluate_clustering(raw_data[1:num_rows, :], base_assignments, base_centers, dist)
                        base_silhouette_raw = average_silhouette_score(raw_data[1:num_rows, :], base_assignments, base_centers)
                        # Add to results DataFrame
                        push!(results, (DatasetName=dataset_name, proportion=prop, k=k, time_limit=time_limit, distance=dist, case="Base", WCSS=base_wcss, WCSS_RAW=base_wcss_raw, Silhouette=base_silhouette, Silhouette_RAW=base_silhouette_raw))
    
                        # kmeans_optimal cases
                        for warm_start in [false, true]
                            try
                                optimal_result = kmedoids_optimal(data, num_rows, dist, k, time_limit, warm_start)  #used to be: optimal_result = kmeans_optimal(data, num_rows, dist, k, time_limit, warm_start)
                                # kmedoids_optimal(data, num_rows, distance, k_max, time_limit, warm_start, seed=1)
                                optimal_assignments = get_assignments(optimal_result)
                                optimal_centers = get_centers(optimal_result)
                                optimal_wcss = evaluate_clustering(data[1:num_rows, :], optimal_assignments, optimal_centers, dist)
                                optimal_silhouette = average_silhouette_score(data[1:num_rows, :], optimal_assignments, optimal_centers)
    
                                optimal_wcss_raw = evaluate_clustering(raw_data[1:num_rows, :], optimal_assignments, optimal_centers, dist)
                                optimal_silhouette_raw = average_silhouette_score(raw_data[1:num_rows, :], optimal_assignments, optimal_centers)
                                
                                case_name = warm_start ? "Optimal with Warm Start" : "Optimal without Warm Start"
                                push!(results, (DatasetName=dataset_name, proportion=prop, k=k, time_limit=time_limit, distance=dist, case=case_name, WCSS=optimal_wcss, WCSS_RAW=optimal_wcss_raw, Silhouette=optimal_silhouette, Silhouette_RAW=optimal_silhouette_raw))
                            catch e
                                optimal_wcss = -15095
                                optimal_wcss_raw = -15095
                                optimal_silhouette = -15095
                                optimal_silhouette_raw = -15095
                                
                                case_name = warm_start ? "Optimal with Warm Start" : "Optimal without Warm Start"
                                # push!(results, (DatasetName=dataset_name, proportion=prop, k=k, time_limit=time_limit, distance=dist, case=case_name, WCSS=optimal_wcss, WCSS_RAW=optimal_wcss_raw, Silhouette=optimal_silhouette, Silhouette_RAW=optimal_silhouette_raw))
                           
                                end
                        end
                        
                    end
                end
            end
        end
    end

    return results
end


run_clustering_cases (generic function with 1 method)

In [75]:
# abalone = read_data("abalone_z.csv", "normalized")
# similarity_prediction = read_data("similarity_prediction_z.csv", "normalized")

# abalone_raw = read_data("abalone.csv", "processed")
# similarity_prediction_raw = read_data("similarity_prediction.csv", "processed")

# abalone_scaled = read_data("abalone_mm.csv", "scaled")
# similarity_prediction_scaled = read_data("similarity_prediction_mm.csv", "scaled")

In [76]:
# run_clustering_cases(datasets, raw_data, proportions, k_values, time_limits, distances)
df_abalone = run_clustering_cases([("normalized", abalone), ("scaled", abalone_scaled)],
                                  abalone_raw,
                                  [0.25],
                                  [3],
                                  [30],
                                  ["manhattan", "euclidean"])
CSV.write("./model_results/kmedoids_optimal_abalone.csv", df_abalone)

normalized 0.25 3 30 manhattan type=DataFrame
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter TimeLimit to value 30
Set parameter Seed to value 1
Set parameter Seed to value 1
Set parameter TimeLimit to value 30
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter TimeLimit to value 30
Set parameter Seed to value 1
Set parameter Seed to value 1
Set parameter TimeLimit to value 30
normalized 0.25 3 30 euclidean type=DataFrame
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter TimeLimit to value 30
Set parameter Seed to value 1
Set parameter Seed to value 1
Set parameter TimeLimit to value 30
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter TimeLimit to value 30
Set parameter Seed to value 1
Set parameter Seed to value 1
Set parameter TimeLimit to value 30
scaled 0.25 3 30

"./model_results/kmedoids_optimal_abalone.csv"

In [ ]:
# run_clustering_cases(datasets, raw_data, proportions, k_values, time_limits, distances)
df_abalone = run_clustering_cases([("normalized", abalone), ("scaled", abalone_scaled)],
                                  abalone_raw,
                                  [0.10, 0.25, 0.75],
                                  [3, 4, 5],
                                  [30, 90, 180],
                                  ["manhattan", "euclidean"])
CSV.write("./model_results/kmedoids_optimal_abalone.csv", df_abalone)

In [ ]:
# run_clustering_cases(datasets, raw_data, proportions, k_values, time_limits, distances)
df_similarity = run_clustering_cases([("normalized", similarity_prediction), ("scaled", similarity_prediction_scaled)],
                                     similarity_prediction_raw,
                                     [0.10, 0.25, 0.75],
                                     [3, 4, 5],
                                     [30, 90, 180],
                                     ["manhattan", "euclidean"])
CSV.write("./model_results/kmedoids_optimal_similarity.csv", df_similarity)